In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from PIL import Image
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import recall_score

In [2]:
# 데이터셋 클래스 정의
class FoodDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        # 라벨 추출 및 변환
        label = self.dataframe.iloc[idx][['egg', 'chicken', 'shrimp', 'cheese', 'pork', 'crab', 'cream', 'tofu', 'lobster', 'peanut', 'bread']].to_numpy().astype('float32')

        if self.transform:
            image = self.transform(image)

        return image, label

In [3]:
# 데이터 불러오기 및 라벨링
df = pd.read_csv('train_data_set_food.csv')

# 알레르기 유발 재료 리스트
allergens = ['egg', 'chicken', 'shrimp', 'cheese', 'pork', 'crab', 'cream', 'tofu', 'lobster', 'peanut', 'bread']

In [4]:
# 이미지 경로 설정
df['image_path'] = df.apply(lambda row: os.path.join('food_dataset/images', row['food_class'], 'png', row['image_name']), axis=1)

# 전처리 정의
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
# 하이퍼파라미터 조합
learning_rates = [1e-3, 1e-4, 1e-5]
batch_sizes = [64]
epochs = [4, 7, 12]

# 교차 검증
kf = KFold(n_splits=3)
best_recall = 0
best_params = {}

In [6]:
# recall 계산 함수
def calculate_recall(preds, labels):
    # 예측값을 0과 1로 변환
    preds = torch.sigmoid(preds).detach().cpu().numpy()
    preds = np.round(preds)

    # 실제 레이블을 numpy 배열로 변환
    labels = labels.cpu().numpy()

    return recall_score(labels, preds, zero_division = 0, average='macro')

In [7]:
# cross-validation 실행
for lr in learning_rates:
    for batch_size in batch_sizes:
        for epoch_num in epochs:
            recalls = []

            print(f"Starting cross-validation with lr={lr}, batch_size={batch_size}, epoch_num={epoch_num}")
            for fold, (train_index, test_index) in enumerate(kf.split(df)):
                print(f"Starting fold {fold+1}")
                train_df = df.iloc[train_index]
                test_df = df.iloc[test_index]

                train_dataset = FoodDataset(train_df, transform=transform)
                test_dataset = FoodDataset(test_df, transform=transform)

                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

                # 모델 초기화 및 설정
                weights = ResNet18_Weights.IMAGENET1K_V1 if torch.cuda.is_available() else ResNet18_Weights.DEFAULT
                model = resnet18(weights=weights)
                model.fc = nn.Linear(model.fc.in_features, len(allergens))
                criterion = nn.BCEWithLogitsLoss()
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # 모델 학습
                for epoch in range(epoch_num):
                    model.train()
                    print(f"Starting epoch {epoch+1}/{epoch_num} for fold {fold+1}")
                    for i, (inputs, labels) in enumerate(train_loader):
                        optimizer.zero_grad()
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()

                # 모델 평가
                model.eval()
                total_recall = 0
                for i, (inputs, labels) in enumerate(test_loader):
                    outputs = model(inputs)
                    recall = calculate_recall(outputs, labels)
                    total_recall += recall

                recalls.append(total_recall / len(test_loader))

                print(f"Completed fold {fold+1} with average recall: {recalls[-1]}")

            average_recall = np.mean(recalls)
            print(f"Completed cross-validation with average recall: {average_recall}")

            if average_recall > best_recall:
                best_recall = average_recall
                best_params = {'lr': lr, 'batch_size': batch_size, 'epochs': epoch_num}

print(f"Best Recall: {best_recall}")
print(f"Best Parameters: {best_params}")

Starting cross-validation with lr=0.001, batch_size=64, epoch_num=4
Starting fold 1
Starting epoch 1/4 for fold 1
Starting epoch 2/4 for fold 1
Starting epoch 3/4 for fold 1
Starting epoch 4/4 for fold 1
Completed fold 1 with average recall: 0.06755433119069483
Starting fold 2
Starting epoch 1/4 for fold 2
Starting epoch 2/4 for fold 2
Starting epoch 3/4 for fold 2
Starting epoch 4/4 for fold 2
Completed fold 2 with average recall: 0.08976612118323347
Starting fold 3
Starting epoch 1/4 for fold 3
Starting epoch 2/4 for fold 3
Starting epoch 3/4 for fold 3
Starting epoch 4/4 for fold 3
Completed fold 3 with average recall: 0.1596656500065591
Completed cross-validation with average recall: 0.10566203412682913
Starting cross-validation with lr=0.001, batch_size=64, epoch_num=7
Starting fold 1
Starting epoch 1/7 for fold 1
Starting epoch 2/7 for fold 1
Starting epoch 3/7 for fold 1
Starting epoch 4/7 for fold 1
Starting epoch 5/7 for fold 1
Starting epoch 6/7 for fold 1
Starting epoch 7/7 

Starting epoch 2/12 for fold 1
Starting epoch 3/12 for fold 1
Starting epoch 4/12 for fold 1
Starting epoch 5/12 for fold 1
Starting epoch 6/12 for fold 1
Starting epoch 7/12 for fold 1
Starting epoch 8/12 for fold 1
Starting epoch 9/12 for fold 1
Starting epoch 10/12 for fold 1
Starting epoch 11/12 for fold 1
Starting epoch 12/12 for fold 1
Completed fold 1 with average recall: 0.23698292448292446
Starting fold 2
Starting epoch 1/12 for fold 2
Starting epoch 2/12 for fold 2
Starting epoch 3/12 for fold 2
Starting epoch 4/12 for fold 2
Starting epoch 5/12 for fold 2
Starting epoch 6/12 for fold 2
Starting epoch 7/12 for fold 2
Starting epoch 8/12 for fold 2
Starting epoch 9/12 for fold 2
Starting epoch 10/12 for fold 2
Starting epoch 11/12 for fold 2
Starting epoch 12/12 for fold 2
Completed fold 2 with average recall: 0.22501846291953242
Starting fold 3
Starting epoch 1/12 for fold 3
Starting epoch 2/12 for fold 3
Starting epoch 3/12 for fold 3
Starting epoch 4/12 for fold 3
Starting 

In [5]:
train_df = pd.read_csv('train_data_set_food.csv')
test_df = pd.read_csv('test_data_set_food.csv')

# 이미지 경로 설정
train_df['image_path'] = train_df.apply(lambda row: os.path.join('food_dataset/images', row['food_class'], 'png', row['image_name']), axis=1)
test_df['image_path'] = test_df.apply(lambda row: os.path.join('food_dataset/images', row['food_class'], 'png', row['image_name']), axis=1)

# 데이터셋 객체 생성 및 DataLoader 정의
train_dataset = FoodDataset(train_df, transform=transform)
test_dataset = FoodDataset(test_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
# 제안 모델 정의
weights = ResNet18_Weights.IMAGENET1K_V1 if torch.cuda.is_available() else ResNet18_Weights.DEFAULT
resnet = models.resnet18(weights=weights)
resnet.fc = nn.Linear(resnet.fc.in_features, len(allergens))

# 모델 초기화
resnet_model = resnet

In [7]:
# 각 알레르기 유발 재료의 출현 빈도 계산
label_counts = train_df[allergens].sum().to_numpy()

# 전체 데이터 수
total_samples = len(train_df)

# 클래스 가중치 계산
class_weights = total_samples / (len(allergens) * label_counts)
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)
# CUDA 사용 시
if torch.cuda.is_available():
    class_weights = class_weights.cuda()

# 가중치 조정 손실 함수 정의
criterion = nn.BCEWithLogitsLoss(weight=class_weights)

# 옵티마이저 정의
# cross-validation 결과: lr=1e-5, epoch=7로 진행
optimizer_resnet = optim.Adam(resnet_model.parameters(), lr=1e-5)
num_epochs = 20

tensor([0.8175, 1.9269, 2.1302, 0.7759, 1.0060, 5.9746, 0.7947, 3.9551, 7.8369,
        5.6455, 0.4155])


In [8]:
# recall 계산 함수
def calculate_recall(preds, labels):
    preds = torch.sigmoid(preds).detach().cpu().numpy()
    preds = np.round(preds)
    labels = labels.cpu().numpy()
    return recall_score(labels, preds, average='macro', zero_division=0)

In [9]:
for epoch in range(num_epochs):
    # ResNet 모델 학습
    resnet_model.train()
    train_loss_resnet = 0.0
    for batch in train_loader:
        inputs, labels = batch
        labels = labels.float()
        
        optimizer_resnet.zero_grad()
        outputs = resnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_resnet.step()
        train_loss_resnet += loss.item()
        
    # ResNet 모델 테스트
    resnet_model.eval()
    test_loss_resnet = 0.0
    total_recall = 0.0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = resnet_model(inputs)
            loss = criterion(outputs, labels)
            test_loss_resnet += loss.item()

            # 재현율 계산
            recall = calculate_recall(outputs, labels)
            total_recall += recall

    average_recall = total_recall / len(test_loader)
    
    # 에포크별 평균 손실 계산 및 출력
    train_loss_resnet /= len(train_loader)
    test_loss_resnet /= len(test_loader)
    
    print(f'Epoch {epoch+1}, Train Loss: {train_loss_resnet:.4f}, Test Loss: {test_loss_resnet:.4f}, Average Recall: {average_recall}')

Epoch 1, Train Loss: 0.6963, Test Loss: 0.3035, Average Recall: 0.08105476004878061
Epoch 2, Train Loss: 0.2591, Test Loss: 0.2347, Average Recall: 0.19501039987194563
Epoch 3, Train Loss: 0.2106, Test Loss: 0.2105, Average Recall: 0.28154804979800285
Epoch 4, Train Loss: 0.1829, Test Loss: 0.1980, Average Recall: 0.32057222630988874
Epoch 5, Train Loss: 0.1616, Test Loss: 0.1935, Average Recall: 0.3651513836843989
Epoch 6, Train Loss: 0.1434, Test Loss: 0.1915, Average Recall: 0.3831109090300525
Epoch 7, Train Loss: 0.1266, Test Loss: 0.1936, Average Recall: 0.3943238925797981
Epoch 8, Train Loss: 0.1116, Test Loss: 0.1959, Average Recall: 0.40300512039440933
Epoch 9, Train Loss: 0.0980, Test Loss: 0.2026, Average Recall: 0.40464492163589755
Epoch 10, Train Loss: 0.0861, Test Loss: 0.2068, Average Recall: 0.4248428751777873
Epoch 11, Train Loss: 0.0751, Test Loss: 0.2152, Average Recall: 0.42926041192342074
Epoch 12, Train Loss: 0.0648, Test Loss: 0.2217, Average Recall: 0.43841721680

In [10]:
torch.save(resnet_model.state_dict(), 'new_multi_label_model.pth')